# How to Use Sklearn Pipelines For Ridiculously Neat Code
## Everything I love about Scikit-learn, all in one place
<img src='https://cdn-images-1.medium.com/max/1200/1*fLcObhT_JaC-WD8FkvT3JQ.jpeg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@abhiram2244?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Abhiram Prakash</a>
        on 
        <a href='https://www.pexels.com/photo/man-sitting-on-edge-facing-sunset-915972/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Setup

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Why Do You Need a Pipeline?

Data cleaning and preparation is easily the most time-consuming and boring task in machine learning. All ML algorithms are really fussy, some want normalized or standardized features, some want encoded variables and some want both. Then, there is also the issue of missing values which is always there.

Dealing with them is no fun at all, not to mention the added bonus that comes with repeating the same cleaning operations on all training, validation and test sets. Fortunately, Scikit-learn's `Pipeline` is a major productivity tool to facilitate this process, cleaning up code and collapsing all preprocessing and modeling steps into to a single line of code. Here, check this out:

```python
# Fit lasso regression
pipe_lasso.fit(X_train, y_train)

# Predict on X_Test
preds = pipe_lasso.predict(X_test)

```

Above, `pipe_lasso` is an instance of such pipeline where it fills the missing values in `X_train` as well as feature scale the numerical columns and one-hot encode categorical variables finishing up by fitting Lasso Regression. When you call `.predict` the same steps are applied to `X_test`, which is really awesome. 

Pipelines combine everything I love about Scikit-learn: conciseness, consistency and easy of use. So, without further ado, let me show how you can build your own pipeline in a few minutes.

### Intro to Scikit-learn Pipelines

In this and coming sections, we will build the above `pipe_lasso` pipeline together for the [Ames Housing dataset](https://www.kaggle.com/c/home-data-for-ml-course/data) which is used for an [InClass competition](https://www.kaggle.com/c/home-data-for-ml-course/overview) on Kaggle. The dataset contains 81 variables on almost every aspect of a house and using these, you have to predict the house's price. Let's load the training and test sets:

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
X_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

train.iloc[:, 70:]

Everything except for the last column - `SalePrice` is used as features. Before we do anything, let's divide up the training data into train, validation sets. We will use the final `X_test` set for predictions.

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop('SalePrice', axis=1)
y = train.SalePrice

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=1121218)

In [ ]:
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))

# Drop categorical columns that will not be encoded
X_train = X_train.drop(bad_label_cols, axis=1)
X_valid = X_valid.drop(bad_label_cols, axis=1)

Now, let's do basic exploration of the training set:

In [ ]:
X_train.describe().T.iloc[:10] # All numerical cols

In [ ]:
X_train.describe(include=np.object).T.iloc[:10] # All object cols

In [ ]:
above_0_missing = X_train.isnull().sum() > 0

X_train.isnull().sum()[above_0_missing]

19 features have NaNs. 

In [ ]:
numerical_features = X_train.select_dtypes(include='number').columns.tolist()
print(f'There are {len(numerical_features)} numerical features:', '\n')
print(numerical_features)

In [ ]:
categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()
print(f'There are {len(categorical_features)} categorical features:', '\n')
print(categorical_features)

Now, on to preprocessing. For numeric columns, we first fill the missing values with `SimpleImputer` using the mean and feature scale using `MinMaxScaler`. For categoricals, we will use `SimpleImputer` to fill the missing values with the mode of each column. Most importantly, we do all of these in a pipeline. Let's import everything:

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

We create two small pipelines for both numeric and categorical features:

In [ ]:
numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

> Set `handle_unknown` to `ignore` to skip previously unseen labels. Otherwise, `OneHotEncoder` throws an error if there exists labels in test set that are not in train set.

[`sklearn.pipeline.Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class takes a tuple of transformers for its `steps` argument. Each tuple should have this pattern:

```
('name_of_transformer`, transformer)
```

Then, each tuple is called a *step* containing a transformer like `SimpleImputer` and arbitrary name. Each step will be chained and applied to the passed DataFrame in the given order.

But, these two pipelines are useless if we don't tell which columns they should be applied to. For that, we will use another transformer - `ColumnTransformer`.

### Column Transformer

By default, all `Pipeline` objects have `fit` and `transform` methods which can be used to transform the input array like this:

In [ ]:
numeric_pipeline.fit_transform(X_train.select_dtypes(include='number'))

Above, we are using the new numeric preprocessor on `X_train` using `fit_transform`. We are specifying the columns with `select_dtypes`. But, using the pipelines in this way means we have to call each pipeline separately on selected columns which is not what we want. What we want is to have a single preprocessor that is able to perform both numeric and categorical transformations in a single line of code like this:

```python
full_processor.fit_transform(X_train)
```

To achieve this, we will use `ColumnTransformer` class:

In [ ]:
from sklearn.compose import ColumnTransformer

full_processor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, numerical_features),
    ('category', categorical_pipeline, categorical_features)
])

> Remember that `numerical_features` and `categorical_features` contain the respective names of columns from `X_train`.

Similar to `Pipeline` class, `ColumnTransformer` takes a tuple of transformers. Each tuple should contain an arbitrary step name, the transformer itself and the list of column names that the transformer should be applied to. Here, we are creating a column transformer with 2 steps using both of our numeric and categorical preprocessing pipelines. Now, we can use it to fully transform the `X_train`:

In [ ]:
full_processor.fit_transform(X_train)

Note that most transformers return `numpy` arrays which means index and column names will be dropped. 

Finally, we managed to collapse all preprocessing steps into a single line of code. However, we can go even further. We can combine preprocessing and modeling to have even neater code.

### Final Pipeline With an Estimator

Adding an estimator (model) to a pipeline is as easy as creating a new pipeline which contains the above column transformer and the model itself. Let's import and instantiate `LassoRegression` and add it to a new pipeline with the `full_processor`:

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

lasso = Lasso(alpha=0.1)

lasso_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', lasso)
])

> Warning! The order of steps matter! The estimator should always be the last step for the pipeline to work correctly.

That's it! We can now call `lasso_pipeline` just like we call any other model. When we call `.fit`, the pipeline applies all transformations before fitting an estimator:

In [ ]:
_ = lasso_pipeline.fit(X_train, y_train)

Let's evaluate our base model on the validation set (Remember, we have a separate testing set which we haven't touched so far):

In [ ]:
preds = lasso_pipeline.predict(X_valid)
mean_absolute_error(y_valid, preds)

In [ ]:
lasso_pipeline.score(X_valid, y_valid)

Great, our base pipeline works. Another great thing about pipelines is that they can be treated as any other model. In other words, we can plug it into anywhere where we would use Scikit-learn estimators. So, we will use the pipeline in a grid search to find the optimal hyperparameters in the next section.

### Using Your Pipeline Everywhere

The main hyperparameter for `Lasso` is alpha which can range from 0 to infinity. For simplicity, we will only cross-validate on the values within 0 and 1 with steps of 0.05:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_dict = {'model__alpha': np.arange(0, 1, 0.05)}

search = GridSearchCV(lasso_pipeline, param_dict, 
                      cv=10, 
                      scoring='neg_mean_absolute_error')

_ = search.fit(X_train, y_train)

Now, we can get the best score and parameters for `Lasso`:

In [ ]:
print('Best score:', abs(search.best_score_))

In [ ]:
print('Best alpha:', search.best_params_)

As you can see, best `alpha` is 0.95 which is the very end of our given interval, i. e. \[0, 1) with a step of 0.05. We need to search again in case the best parameter lies in a bigger interval:

In [ ]:
param_dict = {'model__alpha': np.arange(1, 200, 5)}

search = GridSearchCV(lasso_pipeline, param_dict, 
                      cv=10, 
                      scoring='neg_mean_absolute_error')

_ = search.fit(X_train, y_train)

In [ ]:
print('Best score:', abs(search.best_score_))

In [ ]:
print('Best alpha:', search.best_params_)

With best hyperparameters, we get a significant drop in MAE (which is good). Let's redefine our pipeline with `Lasso(alpha=76)`:

In [ ]:
lasso = Lasso(alpha=181)

final_lasso_pipe = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', lasso)
])

Fit it to `X_train`, validate on `X_valid` and submit predictions for the competition using `X_test`:

In [ ]:
_ = final_lasso_pipe.fit(X_train, y_train)
preds = final_lasso_pipe.predict(X_valid)

mean_absolute_error(y_valid, preds)

### Conclusion

In summary, pipelines introduce several advantages to your daily workflow such as compact and fast code, ease of use and in-place modification of multiple steps. In the examples, we used simple Lasso regression but the pipeline we created could be used for virtually any model out there. Go and use it to build something awesome!